In [8]:
import time
import math

# Questão 2

A estratégia de polinômios descrita na Questão 1 pode ser estendida para o ajuste de qualquer combinação linear de funções. Se você deseja ajustar a curva p1f1(x) + p2f2(x) + ... + pkfk(x) = y aos pontos, basta montar uma matriz A onde a coluna i contém os valores de fi(x) para cada ponto X dos dados e repetir o procedimento anterior para obter os valores dos parâmteros pi no vetor P.

A função p1exp(p2x) = y não é linear, pois o parâmetro p2 está dentro da função exponencial e não é um simples multiplicador.

a) Tranforme este problema em um problema linear.

Aplicamos o logaritmo natural em ambos os lados:
   log(y) = log(p1) + p2 * x

Definindo:
   a = log(p1)

O modelo se torna:
    log(y) = a + p2 * x

Este é um modelo linear em relação aos parâmetros a e p2, que pode ser ajustado utilizando técnicas de regressão linear. Após encontrar os valores de a e p2, recuperamos o parâmetro original p1 com:
   p1 = exp(a)

Assim, transformamos o problema original em um problema linear.

b) Escreva uma função ajustaExp(X, Y) que utiliza a função ajusta, da Questão 1, e transofmra a saída P para obter p1 e p2 de p1exp(p2x) = y.

Função ajusta da questão 1:

In [21]:
def ler_pontos(nome_arquivo):
    with open(nome_arquivo, "r") as f:
        n = int(f.readline().strip())
        X, Y = [], []
        for _ in range(n):
            linha = f.readline().strip().split(",")
            X.append(float(linha[0]))
            Y.append(float(linha[1]))
    return X, Y

In [22]:
def poly(X, k):
    A = []
    for x in X:
        linha = []
        for exp in range(k, -1, -1):
            linha.append(x**exp)
        A.append(linha)
    return A

In [23]:
def inv(M):
    n = len(M)
    # Cria uma matriz identidade
    identidade = [[float(i == j) for j in range(n)] for i in range(n)]
    # Faz uma cópia de M para não alterar a matriz original
    mat = [linha[:] for linha in M]

    for i in range(n):
        # Pivô
        piv = mat[i][i]
        if abs(piv) < 1e-12:  # Evita divisões por zero
            # Tenta encontrar linha para trocar
            for r in range(i + 1, n):
                if abs(mat[r][i]) > abs(piv):
                    mat[i], mat[r] = mat[r], mat[i]
                    identidade[i], identidade[r] = identidade[r], identidade[i]
                    piv = mat[i][i]
                    break
        # Normaliza linha i
        for c in range(n):
            mat[i][c] /= piv
            identidade[i][c] /= piv

        # Elimina demais linhas
        for r in range(n):
            if r != i:
                fator = mat[r][i]
                for c in range(n):
                    mat[r][c] -= fator * mat[i][c]
                    identidade[r][c] -= fator * identidade[i][c]

    return identidade

In [24]:
def transposta(M):
    return [list(linha) for linha in zip(*M)]

In [25]:
def matmul(A, B):
    linhasA, colunasA = len(A), len(A[0])
    linhasB, colunasB = len(B), len(B[0])
    # Multiplicação possível apenas se colunasA == linhasB
    C = [[0] * colunasB for _ in range(linhasA)]
    for i in range(linhasA):
        for j in range(colunasB):
            soma = 0
            for k2 in range(colunasA):
                soma += A[i][k2] * B[k2][j]
            C[i][j] = soma
    return C

In [26]:
def ajusta(X, Y, k):
    inicio = time.time()

    A = poly(X, k)
    At = transposta(A)
    # A^T A
    M = matmul(At, A)
    # Inversão
    Minv = inv(M)
    # Converte Y para matriz coluna
    Ycol = [[y] for y in Y]
    # P = inv(A^T A) * A^T * Y
    Pmat = matmul(Minv, matmul(At, Ycol))
    # Transformar a matriz coluna em lista
    P = [row[0] for row in Pmat]

    # Cálculo do SSE
    SSE = 0
    for i in range(len(X)):
        # Predição para o ponto i
        pred = 0
        for j in range(k + 1):
            pred += A[i][j] * P[j]
        SSE += (pred - Y[i]) ** 2

    tempo_calc = time.time() - inicio
    return P, tempo_calc, SSE

Implementação da função ajustaExp(X, Y) para ajustar o modelo
    
    p1*exp(p2*x) = y, utilizando a função ajusta da Questão 1.

Transformamos o problema aplicando o logaritmo natural:

    log(y) = log(p1) + p2 * x, ou seja, usando a regressão linear para:
    log(y) = a + p2 * x, onde a = log(p1).

Após ajustar, obtemos p1 = exp(a).

In [27]:
def ajustaExp(X, Y):
    # Verifica se todos os valores de Y são positivos
    if any(y <= 0 for y in Y):
        raise ValueError(
            "Todos os valores de Y devem ser positivos para aplicar logaritmo."
        )

    # Aplica o logaritmo natural em Y
    logY = [math.log(y) for y in Y]
    # Ajusta um polinômio de grau 1: poly(X,1) gera matriz com [x, 1]
    # Assim, o modelo ajustado será: P[0]*x + P[1] ≈ log(y)
    P, tempo_calc, SSE = ajusta(X, logY, 1)
    # Recupera os parâmetros do modelo original:
    p2 = P[0]  # Coeficiente associado a x
    a = P[1]  # intercepto, sendo log(p1)
    p1 = math.exp(a)  # Recupera p1
    return (p1, p2, tempo_calc, SSE)

In [28]:
# Gerar dados de teste
def gerar_dados(p1, p2, xs):
    return [p1 * math.exp(p2 * x) for x in xs]

In [29]:
# Definir valores de X e calcular Y usando o modelo verdadeiro
X_teste = [0, 1, 2, 3, 4, 5]
p1_verdadeiro = 2
p2_verdadeiro = 0.3
Y_teste = gerar_dados(p1_verdadeiro, p2_verdadeiro, X_teste)

In [30]:
# Chamar a função ajustaExp para obter os parâmetros ajustados
p1_ajustado, p2_ajustado, tempo_calc, SSE = ajustaExp(X_teste, Y_teste)

In [31]:
print("Parâmetros ajustados:")
print("p1 =", p1_ajustado)
print("p2 =", p2_ajustado)
print("Tempo de cálculo:", tempo_calc)
print("SSE:", SSE)

Parâmetros ajustados:
p1 = 1.9999999999999993
p2 = 0.2999999999999998
Tempo de cálculo: 5.650520324707031e-05
SSE: 4.215475462274782e-30


In [32]:
# Opcional: calcular os valores ajustados para comparação
Y_ajustados = [p1_ajustado * math.exp(p2_ajustado * x) for x in X_teste]
print("\nValores reais vs ajustados:")
for x, y_real, y_aj in zip(X_teste, Y_teste, Y_ajustados):
    print(f"x={x}, y_real={y_real:.4f}, y_ajustado={y_aj:.4f}")


Valores reais vs ajustados:
x=0, y_real=2.0000, y_ajustado=2.0000
x=1, y_real=2.6997, y_ajustado=2.6997
x=2, y_real=3.6442, y_ajustado=3.6442
x=3, y_real=4.9192, y_ajustado=4.9192
x=4, y_real=6.6402, y_ajustado=6.6402
x=5, y_real=8.9634, y_ajustado=8.9634
